<a href="https://colab.research.google.com/github/jbenasuli/final_project/blob/database/AmazonDatabaseSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
#1 Selected furniture data
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_furniture = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df_furniture.show()
print((df_furniture.count(), len(df_furniture.columns)))

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [ ]:
#2 Selected personal care applianaces
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_personal_care = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
print((df_personal_care.count(), len(df_personal_care.columns)))

(878094, 15)


In [ ]:
#3 Selected office products
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_office = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
print((df_office.count(), len(df_office.columns)))

(3520528, 15)


In [ ]:
#4 Selected apparel products
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_apparel = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
print((df_apparel.count(), len(df_apparel.columns)))

(9426861, 15)


In [ ]:
#5 Selected home products
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_home = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
print((df_home.count(), len(df_home.columns)))

(15648420, 15)


In [ ]:
#6 Merge databases
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

df_merge = unionAll(df_furniture, df_personal_care, df_office, df_apparel, df_home)
df_merge.show()

print((df_merge.count(), len(df_merge.columns)))

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [ ]:
#7 Only considering verified purchases
df_merge = df_merge.filter(df_merge.verified_purchase == "Y")

In [ ]:
#8 Drop the non-beneficial columns
columns_to_drop = ["marketplace","product_id","product_title","review_date","review_id","product_parent","star_rating","helpful_votes","total_votes","vine","review_headline","review_body","verified_purchase"]

df_merge = df_merge.drop(*columns_to_drop)
df_merge.show()

print((df_merge.count(), len(df_merge.columns)))

+-----------+----------------+
|customer_id|product_category|
+-----------+----------------+
|   24509695|       Furniture|
|   34731776|       Furniture|
|    1272331|       Furniture|
|   45284262|       Furniture|
|   18311821|       Furniture|
|   42943632|       Furniture|
|   43157304|       Furniture|
|   51918480|       Furniture|
|   14522766|       Furniture|
|   43054112|       Furniture|
|   26622950|       Furniture|
|   17988940|       Furniture|
|   18444952|       Furniture|
|   16937084|       Furniture|
|   23665632|       Furniture|
|    4110125|       Furniture|
|     107621|       Furniture|
|    2415090|       Furniture|
|   48285966|       Furniture|
|   33228559|       Furniture|
+-----------+----------------+
only showing top 20 rows

(26788263, 2)


In [ ]:
#9 Get datatypes
df_merge.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- product_category: string (nullable = true)



In [ ]:
summary_table = df_merge.groupby("customer_id").pivot("product_category").count()
summary_table.show()

+-----------+-------+---------+----+---------------+------------------------+
|customer_id|Apparel|Furniture|Home|Office Products|Personal_Care_Appliances|
+-----------+-------+---------+----+---------------+------------------------+
|   25626507|   null|        5|null|           null|                    null|
|     357089|   null|        5|   2|             12|                    null|
|   48961358|      4|       10|null|              3|                    null|
|   41940521|      4|        5|null|           null|                    null|
|   42652255|   null|        5|null|           null|                    null|
|   23126059|   null|        5|null|           null|                    null|
|   50904584|      2|        5|null|           null|                    null|
|   19717636|      4|        5|null|           null|                    null|
|   49976101|   null|       10|null|           null|                    null|
|     130557|   null|     null|null|              3|            

In [ ]:
# Replace null for 0
summary_table = summary_table.na.fill(value=0)
summary_table.show()

+-----------+-------+---------+----+---------------+------------------------+
|customer_id|Apparel|Furniture|Home|Office Products|Personal_Care_Appliances|
+-----------+-------+---------+----+---------------+------------------------+
|   25626507|      0|        5|   0|              0|                       0|
|     357089|      0|        5|   2|             12|                       0|
|   48961358|      4|       10|   0|              3|                       0|
|   41940521|      4|        5|   0|              0|                       0|
|   42652255|      0|        5|   0|              0|                       0|
|   23126059|      0|        5|   0|              0|                       0|
|   50904584|      2|        5|   0|              0|                       0|
|   19717636|      4|        5|   0|              0|                       0|
|   49976101|      0|       10|   0|              0|                       0|
|     130557|      0|        0|   0|              3|            

In [ ]:
print((summary_table.count(), len(summary_table.columns)))

(6407118, 6)


In [ ]:
# Select smaller amount of data for pandas conversion
summary_table_sample = summary_table.sample(False, 0.05, 42)
summary_table_sample.show()

print((summary_table_sample.count(), len(summary_table_sample.columns)))

+-----------+-------+---------+----+---------------+------------------------+
|customer_id|Apparel|Furniture|Home|Office Products|Personal_Care_Appliances|
+-----------+-------+---------+----+---------------+------------------------+
|   10427817|      0|        0|   0|              3|                       0|
|   22120629|     18|        0|   7|              0|                       0|
|   28496071|      2|        0|   2|              0|                       0|
|   39426368|      2|        0|   1|              0|                       0|
|   43056892|     10|        5|   3|              6|                       0|
|   52907072|      2|        0|   0|              0|                       0|
|   11118822|      0|        0|   1|              3|                       0|
|   44800134|      0|        0|   2|              0|                       0|
|   43255733|      0|        5|   0|              0|                       0|
|    7681812|     14|        0|   1|              6|            

In [ ]:
#14 Convert Pyspark dataframe into panda df
sample_df = summary_table_sample.toPandas()
sample_df

,customer_id,Apparel,Furniture,Home,Office Products,Personal_Care_Appliances
0,10427817,0,0,0,3,0
1,22120629,18,0,7,0,0
2,28496071,2,0,2,0,0
3,39426368,2,0,1,0,0
4,43056892,10,5,3,6,0
...,...,...,...,...,...,...
319926,43945253,4,0,0,0,0
319927,48498340,2,0,0,0,0
319928,11157509,0,0,1,0,0
319929,28731318,0,0,1,0,0


In [ ]:
# Download to csv to use jupyter notebooks since hv.plot is not available in pyspark
from google.colab import drive
drive.mount('/drive', force_remount=True)
sample_df.to_csv('/drive/My Drive/Segmentation/sample_df_1.csv')

Mounted at /drive
